In [25]:
import tensorflow as tf
import yaml
import os
import PIL
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [26]:
# dataset_util (from object_detection.utils)
# not available for tensorflow versions below 1.4
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Utility functions for creating TFRecord data sets."""

import tensorflow as tf


def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def read_examples_list(path):
  """Read list of training or validation examples.

  The file is assumed to contain a single example per line where the first
  token in the line is an identifier that allows us to find the image and
  annotation xml for that example.

  For example, the line:
  xyz 3
  would allow us to find files xyz.jpg and xyz.xml (the 3 would be ignored).

  Args:
    path: absolute path to examples list file.

  Returns:
    list of example identifiers (strings).
  """
  with tf.gfile.GFile(path) as fid:
    lines = fid.readlines()
  return [line.strip().split(' ')[0] for line in lines]


def recursive_parse_xml_to_dict(xml):
  """Recursively parses XML contents to python dict.

  We assume that `object` tags are the only ones that can appear
  multiple times at the same level of a tree.

  Args:
    xml: xml tree obtained by parsing XML file contents using lxml.etree

  Returns:
    Python dictionary holding XML contents.
  """
  if not xml:
    return {xml.tag: xml.text}
  result = {}
  for child in xml:
    child_result = recursive_parse_xml_to_dict(child)
    if child.tag != 'object':
      result[child.tag] = child_result[child.tag]
    else:
      if child.tag not in result:
        result[child.tag] = []
      result[child.tag].append(child_result[child.tag])
  return {xml.tag: result}

In [27]:
%matplotlib inline

In [28]:
# Label dictionary for training and testing data
LABEL_DICT =  {
    "Green" : 1,
    "Red" : 2,
    "Yellow" : 3,
    "off" : 4,
    }

In [29]:
# Label files for datasets
bosch_train_yaml = 'data/bosch_train/train.yaml'               # 5093 labels for .png images
sim_train_yaml   = 'data/sdce_sim/sim_data_annotations.yaml'   # 277 labels for .jpg images
real_train_yaml  = 'data/sdce_real/real_data_annotations.yaml' # 159 labels .jpg images

# Directories for datasets
bosch_train_dir  = os.path.dirname(bosch_train_yaml)
sim_train_dir    = os.path.dirname(sim_train_yaml)
real_train_dir   = os.path.dirname(real_train_yaml)

print("Dataset directories:")
print(bosch_train_dir)
print(sim_train_dir)
print(real_train_dir)

Dataset directories:
data/bosch_train
data/sdce_sim
data/sdce_real


In [30]:
# Load label index information from yaml files
bosch_index = yaml.load(open(bosch_train_yaml, 'rb').read())
sim_index   = yaml.load(open(sim_train_yaml, 'rb').read())
real_index  = yaml.load(open(real_train_yaml, 'rb').read())

In [31]:
# Helper function to determine whether bosch box labels are "good" (usable)
def good_bosch_record(boxes):
    
    # bad if no boxes included
    if not boxes:
        return False
    
    # bad if one of the boxes too small or false box label in one of the boxes
    for box in boxes:
        width  = box['x_max'] - box['x_min']
        height = box['y_max'] - box['y_min']    
        if height < 10 or width < 5 or box['label'] not in LABEL_DICT:
            return False
        
    return True

In [32]:
# Select appropriate samples from large Bosch dataset
numel_good = 0
for record in bosch_index:
    if good_bosch_record(record['boxes']):
        numel_good += 1
        
print("bosch records: {}/{} satisfy our condition".format(numel_good, len(bosch_index)))

bosch records: 1195/5093 satisfy our condition


In [33]:
# Reduce the bosch label index to the "good" records, convert .png images to .jpg and move samples to jpg folder 
bosch_reduced_index = [record.copy() for record in bosch_index if good_bosch_record(record['boxes'])]
# Previous subdirectory
sub_prev = '/' + 'rgb' + '/'
# Target subdirectory
sub_jpg  = '/' + 'jpg' + '/'

# Convert Bosch record images in folder rgb and save to folder jpg
i_record = 0
for record in bosch_reduced_index:
    new_relative_path = os.path.splitext(record['path'])[0] + '.jpg'
    new_relative_path = new_relative_path.replace(sub_prev, sub_jpg, 1)    
    source_path = os.path.normpath(os.path.join(bosch_train_dir, record['path']))
    target_path = os.path.normpath(os.path.join(bosch_train_dir, new_relative_path))
    
    if not os.path.exists(target_path):
        # export jpg file
        new_dir = os.path.dirname(target_path)
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
    
        img = PIL.Image.open(source_path)
        # img = img.resize((1067, 600), PIL.Image.ANTIALIAS)
        img.save(target_path)
        
    record['path'] = new_relative_path
    bosch_reduced_index[i_record]['path'] = new_relative_path # added to additionally adapt the reduced index saved as a new yaml dump afterwards
    i_record += 1

In [34]:
# Save reduced bosch index as yaml
bosch_good_yaml = os.path.join(bosch_train_dir, 'train_good.yaml')
with open(bosch_good_yaml, 'w') as fid:
    yaml.dump(bosch_reduced_index, fid)

In [35]:
# Convert data labels to bosch format 
def get_bosch_record_with_relative_path(record, base):
    if 'filename' in record:
        # this is a yaml record from the sdce dataset with labels to be converted to Bosch format
        boxes = [{'label': a['class'], 
                  'x_min': a['xmin'],
                  'x_max': a['xmin'] + a['x_width'],
                  'y_min': a['ymin'],
                  'y_max': a['ymin'] + a['y_height']} for a in record['annotations']]
        
        return {'boxes': boxes,
                'path': os.path.normpath(os.path.join(base, record['filename']))}
    
    else:
        # this is a yaml record from the Bosch dataset with labels in Bosch format
        ret = record.copy()
        ret['path'] = os.path.normpath(os.path.join(base, record['path']))
        
        return ret

In [36]:
# Simulated sdce dataset (277 labels for .jpg images)
sim_set   = [get_bosch_record_with_relative_path(r, sim_train_dir) for r in sim_index]

# Real sdce dataset (159 labels .jpg images)
real_set  = [get_bosch_record_with_relative_path(r, real_train_dir) for r in real_index]

# Bosch dataset (5093 labels for .png images converted to .jpg)
bosch_set = [get_bosch_record_with_relative_path(r, bosch_train_dir) for r in bosch_reduced_index]

In [37]:
# Dataset visualization function
import matplotlib.patches as patches

def plot_dataset(dataset, dataset_name = 'dataset'):

    print("Dataset {} ({} samples)\n".format(dataset_name, np.size(dataset)))
    for i_img in range(len(dataset)):
        print("{}[{}]:\n{}".format(dataset_name, i_img, dataset[i_img])) #print("dataset[{}]:\n{}".format(i_img, dataset[i_img]))
        img_path = dataset[i_img]['path'] #bosch_train_dir + bosch_reduced_index[i_img]['path'][1:]
        image    = plt.imread(img_path)    

        fig,ax = plt.subplots(1)
        fig.set_size_inches(12, 6)
        ax.imshow(image)
        for i_rect in range (len(dataset[i_img]['boxes'])):
            x_min = dataset[i_img]['boxes'][i_rect]['x_min']
            y_min = dataset[i_img]['boxes'][i_rect]['y_min']
            dx    = dataset[i_img]['boxes'][i_rect]['x_max'] - dataset[i_img]['boxes'][i_rect]['x_min']
            dy    = dataset[i_img]['boxes'][i_rect]['y_max'] - dataset[i_img]['boxes'][i_rect]['y_min']
            rect_color = 'blue'
            if dataset[i_img]['boxes'][i_rect]['label'] == 'Red':
                rect_color = 'red'
            elif dataset[i_img]['boxes'][i_rect]['label'] == 'Yellow':
                rect_color = 'yellow'
            elif dataset[i_img]['boxes'][i_rect]['label'] == 'Green':
                rect_color = 'green'        
            rect = patches.Rectangle((x_min,y_min),dx,dy,linewidth=3,edgecolor=rect_color,facecolor='none')
            ax.add_patch(rect)

        plt.show()

In [38]:
# Dataset visualization (uncomment relevant line, many plots!)
#plot_dataset(sim_set  , 'sim_set')
#plot_dataset(real_set , 'real_set')
#plot_dataset(bosch_set, 'bosch_set')

In [39]:
# Create shuffled training and test sets
test_size    = 0.2 # proportion of testing samples in relation to whole dataset (rest: training)
random_state = 42 # guaranteed to have the same random sequence by using the same seed number

# Simulated sdce dataset
sim_train_set, sim_test_set = train_test_split(
    sim_set, test_size=test_size, random_state=random_state)

# Real sdce dataset
real_train_set, real_test_set = train_test_split(
    real_set, test_size=test_size, random_state=random_state)

# Bosch dataset
bosch_train_set, bosch_test_set = train_test_split(
    bosch_set, test_size=test_size, random_state=random_state)

# Global train and test set consisting of Simulated sdce dataset, Real sdce dataset and Bosch dataset
train_set = shuffle(sim_train_set + real_train_set + bosch_train_set, random_state=random_state)
test_set  = shuffle(sim_test_set + real_test_set + bosch_test_set, random_state=random_state)

# Simulation train and test set consisting of Simulated sdce dataset
train_set_sim_res = shuffle(sim_train_set, random_state=random_state)
test_set_sim_res  = shuffle(sim_test_set, random_state=random_state)

# Real train and test set consisting of Real sdce dataset and Bosch dataset
train_set_real_res = shuffle(real_train_set + bosch_train_set, random_state=random_state)
test_set_real_res  = shuffle(real_test_set + bosch_test_set, random_state=random_state)

In [40]:
# Create tensorflow record files for training and testing
def create_tf_record(record):
    image_path = record['path']
    
    with tf.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()
    
    img    = PIL.Image.open(image_path)
    width  = img.size[0]
    height = img.size[1]
    
    # relative boxes
    x_min = [float(box['x_min']) / width  for box in record['boxes']]
    x_max = [float(box['x_max']) / width  for box in record['boxes']]
    y_min = [float(box['y_min']) / height for box in record['boxes']]
    y_max = [float(box['y_max']) / height for box in record['boxes']]
    
    text_label = [box['label'].encode() for box in record['boxes']]
    num_label = [int(LABEL_DICT[box['label']]) for box in record['boxes']]
    
    image_format = os.path.splitext(image_path)[1].encode()
    image_path   = image_path.encode()
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(image_path),
        'image/source_id': bytes_feature(image_path),
        'image/encoded': bytes_feature(encoded_image),
        'image/format': bytes_feature(image_format),
        'image/object/bbox/xmin': float_list_feature(x_min),
        'image/object/bbox/xmax': float_list_feature(x_max),
        'image/object/bbox/ymin': float_list_feature(y_min),
        'image/object/bbox/ymax': float_list_feature(y_max),
        'image/object/class/text': bytes_list_feature(text_label),
        'image/object/class/label': int64_list_feature(num_label),
    }))

    return tf_example

In [41]:
# Helper function to write training and testing records
def record_writer(output_file, record_sets):
    writer = tf.python_io.TFRecordWriter(output_file)
    
    for record in record_sets:
        tf_example = create_tf_record(record)
        writer.write(tf_example.SerializeToString())
        
    writer.close()

In [42]:
# Create overall training and testing record
record_writer('data/tl_srb_training.record', train_set)
record_writer('data/tl_srb_testing.record', test_set)

# Create training and testing record for simulation data
record_writer('data/tl_sim_training.record', train_set_sim_res)
record_writer('data/tl_sim_testing.record', test_set_sim_res)

# Create training and testing record real data (sdce and Bosch)
record_writer('data/tl_real_training.record', train_set_real_res)
record_writer('data/tl_real_testing.record', test_set_real_res)

In [50]:
# Train your model (done on server)